# 55 Public, Cleveland Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each scenario separately. These files will be used in the Jupyter Notebook.

3. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion

4. Rerun batch sims in EnergyPlus v9.5.
5. Add scenario names, elevations, and glazing systems to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
resgas_cost = 1.16
reselec_cost = 0.1238

comgas_cost = 0.56
comelec_cost = 0.0972

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.93

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

#Electricity (lb/MWh) 1,436.6 (lb/kWh) 1.4366
####Electricity (lb/MMBtu) 421.03 (lb/kbtu).42103 ####
####Natural Gas (Building Used, lb/MMBtu) 145.66 (lb/kBtu .14566) ####
#Renewable Natural Gas (Building Used, lb/MMBtu) 50.22
#No.2 Fuel Oil (lb/MMBtu) 195.64
#Propane (lb/MMBtu) 168.50
#Renewable Propane (lb/MMBtu) 46.89
#Natural Gas (mCHP NG Engine Used, lb/MMBtu) 176.24
#Natural Gas (mCHP Fuel Cell Used, lb/MMBtu) 142.30

gas_co2 = 0.14566
elec_co2 = 0.42103

In [5]:
# Zone size for EUI

sf = 329.55

In [6]:
# Lighting energy reduction, assumption that LEDs are being used.

lighting_redux = .3

---

## Combine CSV Files in Directory Commercial Electric Heating

In [7]:
# Create variable for files in directory
files = [f for f in os.listdir("data/cle/ComElecElec/") if f.endswith(".csv")]

# files

In [8]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/cle/ComElecElec/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
all_ComElec

# 105120  rows, 108 columns

,Scenario,Elevation,GlazingSystem,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),COMFENZONE PEOPLE 1:People Occupant Count [](Hourly),COMFENZONE:Zone Lights Electricity Energy [J](Hourly),COMFENZONE:Zone Windows Total Heat Gain Rate [W](Hourly),COMFENZONE:Zone Windows Total Heat Gain Energy [J](Hourly),EXTWALL:Surface Outside Face Sunlit Fraction [](Hourly),...,COMFENZONE PEOPLE 1:Zone Thermal Comfort Fanger Model PPD [%](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 90% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 80% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature [C](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature [C](Hourly),COMFENZONE:Zone Cooling Setpoint Not Met Time [hr](Hourly),SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly),SYS-1:Air System NaturalGas Energy [J](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly),SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly)
0,cle_com_elec_10,E,Exist_LowE,01/01 01:00:00,-1.2000,0.0,11869.19907,0.0,0.0,0.0,...,77.977719,-1.0,-1.0,-0.886806,-1.0,0.0,1.533456e+06,0,6.662803e+05,0.0
1,cle_com_elec_10,E,Exist_LowE,01/01 02:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,74.840990,-1.0,-1.0,-0.886806,-1.0,0.0,1.787081e+06,0,8.883738e+05,0.0
2,cle_com_elec_10,E,Exist_LowE,01/01 03:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,77.609970,-1.0,-1.0,-0.886806,-1.0,0.0,1.437250e+06,0,6.662803e+05,0.0
3,cle_com_elec_10,E,Exist_LowE,01/01 04:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,74.922504,-1.0,-1.0,-0.886806,-1.0,0.0,1.860285e+06,0,8.883738e+05,0.0
4,cle_com_elec_10,E,Exist_LowE,01/01 05:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,77.471419,-1.0,-1.0,-0.886806,-1.0,0.0,1.584762e+06,0,6.662803e+05,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,cle_com_elec_09,N,Exist_LowE,12/31 20:00:00,3.4875,0.0,11869.19907,0.0,0.0,0.0,...,84.124350,-1.0,-1.0,-0.714479,-1.0,0.0,0.000000e+00,0,0.000000e+00,0.0
8756,cle_com_elec_09,N,Exist_LowE,12/31 21:00:00,3.5250,0.0,11869.19907,0.0,0.0,0.0,...,76.344404,-1.0,-1.0,-0.714479,-1.0,0.0,1.276364e+06,0,1.406848e+06,0.0
8757,cle_com_elec_09,N,Exist_LowE,12/31 22:00:00,3.6750,0.0,11869.19907,0.0,0.0,0.0,...,74.403663,-1.0,-1.0,-0.714479,-1.0,0.0,5.762689e+05,0,1.406848e+06,0.0
8758,cle_com_elec_09,N,Exist_LowE,12/31 23:00:00,4.2125,0.0,11869.19907,0.0,0.0,0.0,...,79.769622,-1.0,-1.0,-0.714479,-1.0,0.0,7.915104e+05,0,7.034239e+05,0.0


In [9]:
#list(all_ComElec.columns.values)

In [10]:
# Get wanted columns
all_ComElec1 = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

#all_ComElec1

In [11]:
all_ComElec1 = all_ComElec1.dropna().reset_index(drop=True)

#all_ComElec1

-----

## Get the Wanted Data (columns)

In [12]:
# Rename Columns
all_ComElec2 = all_ComElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

#all_ComElec2

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Add EUI (site) column


In [13]:
# Copy dataframe to modify and leave original df intact
get_annual_ComElec = all_ComElec2.copy(deep=True)

#get_annual_ComElec

In [14]:
# Convert Joules to KBtu in dataframe
get_annual_ComElec["FanEnergy(kBtu)"] = get_annual_ComElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingElecEnergy(kBtu)"] = get_annual_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingGasEnergy(kBtu)"] = get_annual_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["CoolingEnergy(kBtu)"] = get_annual_ComElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ComElec["LightingEnergy(kBtu)"] = (get_annual_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_ComElec

In [15]:
# Drop columns
get_annual_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ComElec

In [16]:
# Get data per sf (energy/sf)
get_annual_ComElec["FanEnergy(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ComElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ComElec["LightingEnergy(kBtu/sf)"] = (get_annual_ComElec["LightingEnergy(kBtu)"]) / sf

In [17]:
# Establish EUI (energy/sf)
get_annual_ComElec["EUI(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +  
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
                                                       
#get_annual_ComElec

In [18]:
# Do carbon calculation using variable (lb/sf)
get_annual_ComElec["CO2_Electric(lb/sf)"] = ((get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
get_annual_ComElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),LightingEnergy(kBtu/sf),EUI(kBtu/sf),CO2_Electric(lb/sf)
0,cle_com_elec_10,01/01 01:00:00,E,Exist_LowE,0.631512,1.453436,0.0,0.0,0.003375,0.001916,0.004410,0.0,0.0,0.00001,0.006337,0.002668
1,cle_com_elec_10,01/01 02:00:00,E,Exist_LowE,0.842016,1.693826,0.0,0.0,0.003375,0.002555,0.005140,0.0,0.0,0.00001,0.007705,0.003244
2,cle_com_elec_10,01/01 03:00:00,E,Exist_LowE,0.631512,1.362250,0.0,0.0,0.003375,0.001916,0.004134,0.0,0.0,0.00001,0.006060,0.002552
3,cle_com_elec_10,01/01 04:00:00,E,Exist_LowE,0.842016,1.763210,0.0,0.0,0.003375,0.002555,0.005350,0.0,0.0,0.00001,0.007916,0.003333
4,cle_com_elec_10,01/01 05:00:00,E,Exist_LowE,0.631512,1.502064,0.0,0.0,0.003375,0.001916,0.004558,0.0,0.0,0.00001,0.006484,0.002730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,cle_com_elec_09,12/31 20:00:00,N,Exist_LowE,0.000000,0.000000,0.0,0.0,0.003375,0.000000,0.000000,0.0,0.0,0.00001,0.000010,0.000004
105116,cle_com_elec_09,12/31 21:00:00,N,Exist_LowE,1.333435,1.209759,0.0,0.0,0.003375,0.004046,0.003671,0.0,0.0,0.00001,0.007727,0.003253
105117,cle_com_elec_09,12/31 22:00:00,N,Exist_LowE,1.333435,0.546198,0.0,0.0,0.003375,0.004046,0.001657,0.0,0.0,0.00001,0.005714,0.002406
105118,cle_com_elec_09,12/31 23:00:00,N,Exist_LowE,0.666717,0.750207,0.0,0.0,0.003375,0.002023,0.002276,0.0,0.0,0.00001,0.004310,0.001815


## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier
Add cost multiplier, variables defined above

In [19]:
# Get annual costs
get_annual_costs_ComElec = all_ComElec2.copy(deep=True)

In [20]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ComElec["FanCost($)"] = (get_annual_costs_ComElec["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs_ComElec["HeatingElecCost($)"] = ((get_annual_costs_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["HeatingGasCost($)"] = ((get_annual_costs_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["CoolingCost($)"] = (get_annual_costs_ComElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ComElec["LightingCost($)"] = (get_annual_costs_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_costs

In [21]:
# Drop columns
get_annual_costs_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs_ComElec

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [22]:
# Combine data for energy and cost
allAnnualDataComElec = pd.merge(get_annual_ComElec, get_annual_costs_ComElec)

#allAnnualDataComElec

In [23]:
# Add energy and cost totals
totalComEnergy = allAnnualDataComElec["FanEnergy(kBtu)"] + allAnnualDataComElec["HeatingElecEnergy(kBtu)"] + allAnnualDataComElec["LightingEnergy(kBtu)"] + allAnnualDataComElec["CoolingEnergy(kBtu)"]
        
allAnnualDataComElec["AnnualEnergy(kBtu)"] = totalComEnergy

totalComCost = allAnnualDataComElec["FanCost($)"] +  allAnnualDataComElec["HeatingElecCost($)"] +  allAnnualDataComElec["LightingCost($)"] +   allAnnualDataComElec["CoolingCost($)"]
        
allAnnualDataComElec["AnnualCost($)"] = totalComCost

allAnnualDataComElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),FanEnergy(kBtu/sf),...,LightingEnergy(kBtu/sf),EUI(kBtu/sf),CO2_Electric(lb/sf),FanCost($),HeatingElecCost($),HeatingGasCost($),CoolingCost($),LightingCost($),AnnualEnergy(kBtu),AnnualCost($)
0,cle_com_elec_10,01/01 01:00:00,E,Exist_LowE,0.631512,1.453436,0.0,0.0,0.003375,0.001916,...,0.00001,0.006337,0.002668,0.017990,0.008139,0.0,0.0,0.003375,2.088322,0.029504
1,cle_com_elec_10,01/01 02:00:00,E,Exist_LowE,0.842016,1.693826,0.0,0.0,0.003375,0.002555,...,0.00001,0.007705,0.003244,0.023986,0.009485,0.0,0.0,0.003375,2.539217,0.036846
2,cle_com_elec_10,01/01 03:00:00,E,Exist_LowE,0.631512,1.362250,0.0,0.0,0.003375,0.001916,...,0.00001,0.006060,0.002552,0.017990,0.007629,0.0,0.0,0.003375,1.997137,0.028993
3,cle_com_elec_10,01/01 04:00:00,E,Exist_LowE,0.842016,1.763210,0.0,0.0,0.003375,0.002555,...,0.00001,0.007916,0.003333,0.023986,0.009874,0.0,0.0,0.003375,2.608601,0.037235
4,cle_com_elec_10,01/01 05:00:00,E,Exist_LowE,0.631512,1.502064,0.0,0.0,0.003375,0.001916,...,0.00001,0.006484,0.002730,0.017990,0.008412,0.0,0.0,0.003375,2.136951,0.029776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,cle_com_elec_09,12/31 20:00:00,N,Exist_LowE,0.000000,0.000000,0.0,0.0,0.003375,0.000000,...,0.00001,0.000010,0.000004,0.000000,0.000000,0.0,0.0,0.003375,0.003375,0.003375
105116,cle_com_elec_09,12/31 21:00:00,N,Exist_LowE,1.333435,1.209759,0.0,0.0,0.003375,0.004046,...,0.00001,0.007727,0.003253,0.037985,0.006775,0.0,0.0,0.003375,2.546569,0.048134
105117,cle_com_elec_09,12/31 22:00:00,N,Exist_LowE,1.333435,0.546198,0.0,0.0,0.003375,0.004046,...,0.00001,0.005714,0.002406,0.037985,0.003059,0.0,0.0,0.003375,1.883007,0.044419
105118,cle_com_elec_09,12/31 23:00:00,N,Exist_LowE,0.666717,0.750207,0.0,0.0,0.003375,0.002023,...,0.00001,0.004310,0.001815,0.018992,0.004201,0.0,0.0,0.003375,1.420299,0.026569


---

## Establish Annual Baseline at Existing

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Determine the delta between scenario and base.

In [35]:
# Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "Exist"]

baselineAnnualDataComElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),FanEnergy(kBtu/sf),...,LightingEnergy(kBtu/sf),EUI(kBtu/sf),CO2_Electric(lb/sf),FanCost($),HeatingElecCost($),HeatingGasCost($),CoolingCost($),LightingCost($),AnnualEnergy(kBtu),AnnualCost($)
26280,cle_com_elec_01,01/01 01:00:00,N,Exist,1.682779,2.811337,0.0,0.023699,0.003375,0.005106,...,0.00001,0.013719,0.005776,0.047936,0.015743,0.0,0.000675,0.003375,4.521190,0.067730
26281,cle_com_elec_01,01/01 02:00:00,N,Exist,1.856860,2.319001,0.0,0.000000,0.003375,0.005635,...,0.00001,0.012682,0.005339,0.052895,0.012986,0.0,0.000000,0.003375,4.179236,0.069257
26282,cle_com_elec_01,01/01 03:00:00,N,Exist,1.392645,2.292020,0.0,0.000000,0.003375,0.004226,...,0.00001,0.011191,0.004712,0.039672,0.012835,0.0,0.000000,0.003375,3.688039,0.055882
26283,cle_com_elec_01,01/01 04:00:00,N,Exist,1.856860,2.583973,0.0,0.000000,0.003375,0.005635,...,0.00001,0.013486,0.005678,0.052895,0.014470,0.0,0.000000,0.003375,4.444207,0.070741
26284,cle_com_elec_01,01/01 05:00:00,N,Exist,1.392645,2.482746,0.0,0.000000,0.003375,0.004226,...,0.00001,0.011770,0.004955,0.039672,0.013903,0.0,0.000000,0.003375,3.878765,0.056950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61315,cle_com_elec_04,12/31 20:00:00,W,Exist,1.071490,0.820686,0.0,0.000000,0.003375,0.003251,...,0.00001,0.005752,0.002422,0.030523,0.004596,0.0,0.000000,0.003375,1.895551,0.038494
61316,cle_com_elec_04,12/31 21:00:00,W,Exist,1.558531,0.902658,0.0,0.000000,0.003375,0.004729,...,0.00001,0.007479,0.003149,0.044397,0.005055,0.0,0.000000,0.003375,2.464563,0.052827
61317,cle_com_elec_04,12/31 22:00:00,W,Exist,1.168898,0.677328,0.0,0.000000,0.003375,0.003547,...,0.00001,0.005613,0.002363,0.033298,0.003793,0.0,0.000000,0.003375,1.849601,0.040466
61318,cle_com_elec_04,12/31 23:00:00,W,Exist,1.558531,1.060421,0.0,0.000000,0.003375,0.004729,...,0.00001,0.007957,0.003350,0.044397,0.005938,0.0,0.000000,0.003375,2.622327,0.053710


In [41]:
# Rename columns to indicate baseline to prepare for merge\n"

allBaselineAnnualDataComElec = baselineAnnualDataComElec.rename(columns={"Scenario":"B_Scenario",
                            "Date_Time":"Date_Time",
                            "Elevation":"Elevation",
                            "GlazingSystem":"B_GlazingSystem",
                            "FanEnergy(kBtu)":"B_FanEnergy(kBtu)",
                            "HeatingElecEnergy(kBtu)":"B_HeatingElecEnergy(kBtu)",
                            "HeatingGasEnergy(kBtu)":"B_HeatingGasEnergy(kBtu)",
                            "CoolingEnergy(kBtu)":"B_CoolingEnergy(kBtu)",
                            "LightingEnergy(kBtu)":"B_LightingEnergy(kBtu)",
                            "FanEnergy(kBtu/sf)":"B_FanEnergy(kBtu/sf)",
                            "HeatingElecEnergy(kBtu/sf)":"B_HeatingElecEnergy(kBtu/sf)",
                            "HeatingGasEnergy(kBtu/sf)":"B_HeatingGasEnergy(kBtu/sf)",
                            "CoolingEnergy(kBtu/sf)":"B_CoolingEnergy(kBtu/sf)",
                            "LightingEnergy(kBtu/sf)":"B_LightingEnergy(kBtu/sf)",
                            "AnnualEnergy(kBtu)":"B_AnnualEnergy(kBtu)",
                            "EUI(kBtu/sf)":"B_EUI(kBtu/sf)",
                            "CO2_Electric(lb/sf)":"B_CO2_Electric(lb/sf)",
                            "FanCost($)":"B_FanEnergy($)",
                            "HeatingElecCost($)":"B_HeatingElecCost($)",
                            "HeatingGasCost($)":"B_HeatingGasCost($)",
                            "CoolingCost($)":"B_CoolingCost($)",
                             "LightingCost($)":"B_LightingCost($)",
                            "AnnualCost($)":"B_AnnualCost($)"
                            })
#allBaselineAnnualDataComElec

In [43]:
# Merge the dataframes to include the baseline data.
AnnualWithBaselineCom = pd.merge(allAnnualDataComElec, allBaselineAnnualDataComElec, on=["Date_Time", "Elevation"], how="inner")

AnnualWithBaselineCom

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),FanEnergy(kBtu/sf),...,B_LightingEnergy(kBtu/sf),B_EUI(kBtu/sf),B_CO2_Electric(lb/sf),B_FanEnergy($),B_HeatingElecCost($),B_HeatingGasCost($),B_CoolingCost($),B_LightingCost($),B_AnnualEnergy(kBtu),B_AnnualCost($)
0,cle_com_elec_10,01/01 01:00:00,E,Exist_LowE,0.631512,1.453436,0.0,0.000000,0.003375,0.001916,...,0.00001,0.012601,0.005305,0.026781,0.017792,0.0,0.000909,0.003375,4.152542,0.048857
1,cle_com_elec_02,01/01 01:00:00,E,Exist,0.940145,3.177104,0.0,0.031918,0.003375,0.002853,...,0.00001,0.012601,0.005305,0.026781,0.017792,0.0,0.000909,0.003375,4.152542,0.048857
2,cle_com_elec_06,01/01 01:00:00,E,Exist_Clear,0.785952,1.300328,0.0,0.000000,0.003375,0.002385,...,0.00001,0.012601,0.005305,0.026781,0.017792,0.0,0.000909,0.003375,4.152542,0.048857
3,cle_com_elec_10,01/01 02:00:00,E,Exist_LowE,0.842016,1.693826,0.0,0.000000,0.003375,0.002555,...,0.00001,0.012264,0.005164,0.030607,0.016598,0.0,0.000000,0.003375,4.041712,0.050580
4,cle_com_elec_02,01/01 02:00:00,E,Exist,1.074452,2.963885,0.0,0.000000,0.003375,0.003260,...,0.00001,0.012264,0.005164,0.030607,0.016598,0.0,0.000000,0.003375,4.041712,0.050580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,cle_com_elec_05,12/31 23:00:00,N,Exist_Clear,0.000000,0.000000,0.0,0.000000,0.003375,0.000000,...,0.00001,0.006803,0.002864,0.039672,0.004737,0.0,0.000000,0.003375,2.241938,0.047784
105116,cle_com_elec_09,12/31 23:00:00,N,Exist_LowE,0.666717,0.750207,0.0,0.000000,0.003375,0.002023,...,0.00001,0.006803,0.002864,0.039672,0.004737,0.0,0.000000,0.003375,2.241938,0.047784
105117,cle_com_elec_01,12/31 24:00:00,N,Exist,1.856860,1.436626,0.0,0.000000,0.003375,0.005635,...,0.00001,0.010004,0.004212,0.052895,0.008045,0.0,0.000000,0.003375,3.296861,0.064316
105118,cle_com_elec_05,12/31 24:00:00,N,Exist_Clear,1.680864,0.756547,0.0,0.021268,0.003375,0.005100,...,0.00001,0.010004,0.004212,0.052895,0.008045,0.0,0.000000,0.003375,3.296861,0.064316


In [44]:
list(AnnualWithBaselineCom.columns.values)

['Scenario',
 'Date_Time',
 'Elevation',
 'GlazingSystem',
 'FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)',
 'FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)',
 'FanCost($)',
 'HeatingElecCost($)',
 'HeatingGasCost($)',
 'CoolingCost($)',
 'LightingCost($)',
 'AnnualEnergy(kBtu)',
 'AnnualCost($)',
 'B_Scenario',
 'B_GlazingSystem',
 'B_FanEnergy(kBtu)',
 'B_HeatingElecEnergy(kBtu)',
 'B_HeatingGasEnergy(kBtu)',
 'B_CoolingEnergy(kBtu)',
 'B_LightingEnergy(kBtu)',
 'B_FanEnergy(kBtu/sf)',
 'B_HeatingElecEnergy(kBtu/sf)',
 'B_HeatingGasEnergy(kBtu/sf)',
 'B_CoolingEnergy(kBtu/sf)',
 'B_LightingEnergy(kBtu/sf)',
 'B_EUI(kBtu/sf)',
 'B_CO2_Electric(lb/sf)',
 'B_FanEnergy($)',
 'B_HeatingElecCost($)',
 'B_HeatingGasCost($)',
 'B_CoolingCost($)',
 'B_LightingCost($)',
 'B_AnnualEnergy(kBtu)'

In [38]:
# Add column for the difference from baseline columns (scenario - baseline)
AnnualWithBaselineCom["deltaEnergy(kBtu)"] = AnnualWithBaselineCom["AnnualEnergy(kBtu)"] - AnnualWithBaselineCom["B_AnnualEnergy(kBtu)"]
AnnualWithBaselineCom["deltaEUI(kBtu/sf)"] = AnnualWithBaselineCom["EUI(kBtu/sf)"] - AnnualWithBaselineCom["B_EUI(kBtu/sf)"]
AnnualWithBaselineCom["deltaCO2Elec(lb/sf)"] = AnnualWithBaselineCom["CO2_Electric(lb/sf)"] - AnnualWithBaselineCom["B_CO2_Electric(lb/sf)"]
AnnualWithBaselineCom["deltaCost($)"] = AnnualWithBaselineCom["AnnualCost($)"] - AnnualWithBaselineCom["B_AnnualCost($)"]

AnnualWithBaselineCom

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),FanEnergy(kBtu/sf),...,B_HeatingElecCost($),B_HeatingGasCost($),B_CoolingCost($),B_LightingCost($),B_AnnualEnergy(kBtu),B_AnnualCost($),deltaEnergy(kBtu),deltaEUI(kBtu/sf),deltaCO2Elec(lb/sf),deltaCost($)
0,cle_com_elec_10,01/01 01:00:00,E,Exist_LowE,0.631512,1.453436,0.0,0.000000,0.003375,0.001916,...,0.017792,0.0,0.000909,0.003375,4.152542,0.048857,-2.064220,-0.006264,-0.002637,-0.019354
1,cle_com_elec_02,01/01 01:00:00,E,Exist,0.940145,3.177104,0.0,0.031918,0.003375,0.002853,...,0.017792,0.0,0.000909,0.003375,4.152542,0.048857,0.000000,0.000000,0.000000,0.000000
2,cle_com_elec_06,01/01 01:00:00,E,Exist_Clear,0.785952,1.300328,0.0,0.000000,0.003375,0.002385,...,0.017792,0.0,0.000909,0.003375,4.152542,0.048857,-2.062888,-0.006260,-0.002636,-0.015812
3,cle_com_elec_10,01/01 02:00:00,E,Exist_LowE,0.842016,1.693826,0.0,0.000000,0.003375,0.002555,...,0.016598,0.0,0.000000,0.003375,4.041712,0.050580,-1.502495,-0.004559,-0.001920,-0.013734
4,cle_com_elec_02,01/01 02:00:00,E,Exist,1.074452,2.963885,0.0,0.000000,0.003375,0.003260,...,0.016598,0.0,0.000000,0.003375,4.041712,0.050580,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,cle_com_elec_05,12/31 23:00:00,N,Exist_Clear,0.000000,0.000000,0.0,0.000000,0.003375,0.000000,...,0.004737,0.0,0.000000,0.003375,2.241938,0.047784,-2.238563,-0.006793,-0.002860,-0.044409
105116,cle_com_elec_09,12/31 23:00:00,N,Exist_LowE,0.666717,0.750207,0.0,0.000000,0.003375,0.002023,...,0.004737,0.0,0.000000,0.003375,2.241938,0.047784,-0.821638,-0.002493,-0.001050,-0.021215
105117,cle_com_elec_01,12/31 24:00:00,N,Exist,1.856860,1.436626,0.0,0.000000,0.003375,0.005635,...,0.008045,0.0,0.000000,0.003375,3.296861,0.064316,0.000000,0.000000,0.000000,0.000000
105118,cle_com_elec_05,12/31 24:00:00,N,Exist_Clear,1.680864,0.756547,0.0,0.021268,0.003375,0.005100,...,0.008045,0.0,0.000000,0.003375,3.296861,0.064316,-0.834806,-0.002533,-0.001067,-0.008216


In [ ]:
# Convert NANs to 0
AnnualWithBaselineCom =AnnualWithBaselineCom.fillna(0)

#AnnualWithBaseline

---
## Group and Export Annual Data
Group dataframe by scenario
Export to file

In [ ]:
# Group by scenario
annualDataCom = AnnualWithBaselineCom.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

#annualData.sum().round(2)

In [ ]:
annualDataComFinal = annualDataCom.sum().round(2)

#annualDataComFinal

In [ ]:
# Export grouped file to csv
annualDataComFinal.to_csv("Scraper_Output/cle_annual_com.csv", header=True, index=True)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [ ]:
# Start with renaming df
get_peak_ComElec = get_annual_ComElec
#get_peak_ComElec

In [ ]:
# Add up heat, cool and fans for a total column
get_peak_ComElec["TotalEnergy(kBtu)"] = (get_peak_ComElec["FanEnergy(kBtu)"] +
                                    get_peak_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_peak_ComElec["LightingEnergy(kBtu)"] +
                                    get_peak_ComElec["CoolingEnergy(kBtu)"])

#get_peak_ComElec

In [ ]:
# Get scenario and find the max
scenario1 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_01"]
scenario2 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_02"]
scenario3 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_03"]
scenario4 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_04"]
scenario5 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_05"]
scenario6 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_06"]
scenario7 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_07"]
scenario8 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_08"]
scenario9 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_09"]
scenario10 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_10"]
scenario11 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_11"]
scenario12 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_12"]

In [ ]:
scenarioPeak1 = (scenario1[scenario1["TotalEnergy(kBtu)"] == scenario1["TotalEnergy(kBtu)"].max()])
scenarioPeak2 = (scenario2[scenario2["TotalEnergy(kBtu)"] == scenario2["TotalEnergy(kBtu)"].max()])
scenarioPeak3 = (scenario3[scenario3["TotalEnergy(kBtu)"] == scenario3["TotalEnergy(kBtu)"].max()])
scenarioPeak4 = (scenario4[scenario4["TotalEnergy(kBtu)"] == scenario4["TotalEnergy(kBtu)"].max()])
scenarioPeak5 = (scenario5[scenario5["TotalEnergy(kBtu)"] == scenario5["TotalEnergy(kBtu)"].max()])
scenarioPeak6 = (scenario6[scenario6["TotalEnergy(kBtu)"] == scenario6["TotalEnergy(kBtu)"].max()])
scenarioPeak7 = (scenario7[scenario7["TotalEnergy(kBtu)"] == scenario7["TotalEnergy(kBtu)"].max()])
scenarioPeak8 = (scenario8[scenario8["TotalEnergy(kBtu)"] == scenario8["TotalEnergy(kBtu)"].max()])
scenarioPeak9 = (scenario9[scenario9["TotalEnergy(kBtu)"] == scenario9["TotalEnergy(kBtu)"].max()])
scenarioPeak10 = (scenario10[scenario10["TotalEnergy(kBtu)"] == scenario10["TotalEnergy(kBtu)"].max()])
scenarioPeak11 = (scenario11[scenario11["TotalEnergy(kBtu)"] == scenario11["TotalEnergy(kBtu)"].max()])
scenarioPeak12 = (scenario12[scenario12["TotalEnergy(kBtu)"] == scenario12["TotalEnergy(kBtu)"].max()])

In [ ]:
PeakDemandElec =scenarioPeak1.append([scenarioPeak2, 
                                  scenarioPeak3,
                                  scenarioPeak4,
                                  scenarioPeak5,
                                  scenarioPeak6,
                                  scenarioPeak7,
                                  scenarioPeak8,
                                  scenarioPeak9,
                                  scenarioPeak10,
                                  scenarioPeak11,
                                  scenarioPeak12])

PeakDemandElec

In [ ]:
# Export dataframe  to csv
PeakDemandElec.to_csv("Scraper_Output/cle_peak_com.csv", header=True, index=True)